### Initialize the client

As seen in the "1kg.ipynb" example, we take the following steps to create the client object that will be used to obtain the information we desire and query the serever

In [37]:
import ga4gh.client as client
import matplotlib as plt
c = client.HttpClient("http://1kgenomes.ga4gh.org")

### Make reference to the data from the server

We query the server for the dataset, which is the 1k-genomes dataset.
We follow to access the bases of reference. So to access it we first list the reference sets.

In [50]:
dataset = c.searchDatasets().next()

for variantSet in c.searchVariantSets(dataset.id):
    if variantSet.name == "functional_annotation":
        annotation = variantSet

print(annotation)

annotationSet = c.searchVariantAnnotationSets(variantSetId=annotation.id).next()

print(annotationSet.id)




VariantSet({"name": "functional_annotation", "referenceSetId": "WyJOQ0JJMzciXQ", "id": "WyIxa2dlbm9tZXMiLCJ2cyIsImZ1bmN0aW9uYWxfYW5ub3RhdGlvbiJd", "datasetId": "WyIxa2dlbm9tZXMiXQ", "metadata": [{"info": {}, "description": "", "number": "1", "value": "VCFv4.1", "key": "version", "type": "String", "id": "WyIxa2dlbm9tZXMiLCJ2cyIsImZ1bmN0aW9uYWxfYW5ub3RhdGlvbiIsIm1ldGFkYXRhOnZlcnNpb24iXQ"}, {"info": {}, "description": "Confidence interval around END for imprecise variants", "number": "2", "value": "", "key": "INFO.CIEND", "type": "Integer", "id": "WyIxa2dlbm9tZXMiLCJ2cyIsImZ1bmN0aW9uYWxfYW5ub3RhdGlvbiIsIm1ldGFkYXRhOklORk8uQ0lFTkQiXQ"}, {"info": {}, "description": "Confidence interval around POS for imprecise variants", "number": "2", "value": "", "key": "INFO.CIPOS", "type": "Integer", "id": "WyIxa2dlbm9tZXMiLCJ2cyIsImZ1bmN0aW9uYWxfYW5ub3RhdGlvbiIsIm1ldGFkYXRhOklORk8uQ0lQT1MiXQ"}, {"info": {}, "description": "Source call set.", "number": "1", "value": "", "key": "INFO.CS", "type": "String

In [11]:
def summarizeAnnotations(annotationSetId=None, start=None, end=None, chrom=None):
  termCount = {}
  for annotation in searchVariantAnnotations(annotationSetId, chrom, start, end):
    for eff in annotation.transcriptEffects:
      for term in eff.effects:
        termCount[term] += 1
  return termCount

SyntaxError: invalid syntax (<ipython-input-11-dcb771567dbb>, line 3)

In [51]:
print(summarizeAnnotations(annotationSet.id,start=10000,end=12000,referenceName="1"))

NameError: name 'summarizeAnnotations' is not defined